In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

!pip install -U efficientnet
import efficientnet.tfkeras as efn

     |████████████████████████████████| 50 kB 2.0 MB/s 


In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255,rotation_range=5, width_shift_range=0.1, height_shift_range = 0.1,
                             zoom_range= 0.3, horizontal_flip=True, vertical_flip=True, fill_mode='reflect')
valgen = ImageDataGenerator(rescale = 1.0/255)

testgen = ImageDataGenerator(rescale = 1.0/255)

conv_base = efn.EfficientNetB0(weights="imagenet", include_top=False )

model = models.Sequential()
model.add(conv_base)

model.add(layers.GlobalAveragePooling2D(name="gap"))
model.add(layers.Flatten())
#avoid overfitting
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation="ReLU"))
model.add(layers.Dropout(0.4))
# Set NUMBER_OF_CLASSES to the number of your final predictions.
model.add(layers.Flatten())
model.add(layers.Dense(4, activation="softmax"))

16818176/16804768 [==============================] - 1s 0us/step


In [ ]:
batch_size = 16

reducelr = ReduceLROnPlateau(monitor='val_loss',factor=0.2, patience=8, min_lr=0.00001, mode='max')
earlyStop = EarlyStopping(monitor='val_loss',mode ='min', patience=10)
modelCheckpoint = ModelCheckpoint("/content/drive/MyDrive/218007620_HonorsProject/Model/best_MacenkoB0.h5",monitor='val_acc',
                                  mode='max',save_weights_only=False, save_best_only=True)


TrainGen = datagen.flow_from_directory( "/content/drive/MyDrive/218007620_HonorsProject/Macenko - Train",
                                       target_size=(224, 224), color_mode='rgb',class_mode='categorical',
                                       batch_size=batch_size, shuffle=True, seed=15)

ValGen = valgen.flow_from_directory( "/content/drive/MyDrive/218007620_HonorsProject/Macenko - Validation",
                                       target_size=(224, 224), color_mode='rgb',class_mode='categorical',
                                       batch_size=batch_size, shuffle=False, seed=15)

TestGen = testgen.flow_from_directory("/content/drive/MyDrive/218007620_HonorsProject/Macenko - Test",
                                      target_size=(224, 224),color_mode = 'rgb',batch_size = 1,seed=15, shuffle=False)

Found 1360 images belonging to 4 classes.
Found 180 images belonging to 4 classes.
Found 60 images belonging to 4 classes.


In [ ]:
conv_base.trainable = False
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.0001), metrics=["acc","Precision" ,"Recall"])

model.summary()
model.fit(TrainGen,epochs = 50, verbose=1,
          callbacks=[reducelr, earlyStop,modelCheckpoint],
          validation_data=ValGen)

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name != 'block3':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.0001), metrics=["acc","Precision" ,"Recall"])

model.summary()
model.fit(TrainGen,epochs = 50, verbose=1,
          callbacks=[reducelr, earlyStop,modelCheckpoint],
          validation_data=ValGen)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Functional) (None, None, None, 1280)  4049564   
_________________________________________________________________
gap (GlobalAveragePooling2D) (None, 1280)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
85/85 [==============================] - 230s 3s/step - loss: 1.1872 - acc: 0.4721 - precision: 0.6125 - recall: 0.2441 - val_loss: 1.0588 - val_acc: 0.5444 - val_precision: 0.8000 - val_recall: 0.2889
Epoch 3/50
85/85 [==============================] - 221s 3s/step - loss: 1.0530 - acc: 0.5654 - precision: 0.6912 - recall: 0.3390 - val_loss: 0.9876 - val_acc: 0.5611 - val_precision: 0.7558 - val_recall: 0.3611
Epoch 4/50
85/85 [==============================] - 219s 3s/step - loss: 1.0026 - acc: 0.5912 - precision: 0.7033 - recall: 0.3904 - val_loss: 0.9424 - val_acc: 0.5944 - val_precision: 0.6667 - val_recall: 0.4333
Epoch 5/50
85/85 [==============================] - 219s 3s/step - loss: 0.9510 - acc: 0.6125 - precision: 0.7466 - recall: 0.4397 - val_loss: 0.9139 - val_acc: 0.5944 - val_precision: 0.6880 - val_recall: 0.4778
Epoch 6/50
85/85 [==============================] - 218s 3s/step - loss: 0.8989 - acc: 0.6390 - precision: 0.7494 - recall: 0.4794 - val_loss: 0.874

In [ ]:
from sklearn.metrics import confusion_matrix

import numpy as np

y_pred = model.predict(TestGen, verbose=1)

y_pred = np.argmax(y_pred, axis = 1)

'''test_accuracy = accuracy_score(TestGen.classes, y_pred)
print(test_accuracy)
loss, accuracy, precision, recall = model.evaluate(TestGen, verbose=0)

f1_score = 2*(((precision)*(recall))/(precision)+(recall))

print("test loss, test acc,f1_score, precision,recall", loss, accuracy, f1_score, precision, recall)'''

print(confusion_matrix(TestGen.classes, y_pred))

print(classification_report(TestGen.classes, y_pred))

model.save('/content/drive/MyDrive/Model/best_'+stain+'B0.h5')

from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/218007620_HonorsProject/Model/best_MacenkoB0.h5')

60/60 [==============================] - 97s 2s/step
[[11  2  1  1]
 [ 0 14  1  0]
 [ 0  1 14  0]
 [ 0  0  0 15]]
              precision    recall  f1-score   support

           0       1.00      0.73      0.85        15
           1       0.82      0.93      0.87        15
           2       0.88      0.93      0.90        15
           3       0.94      1.00      0.97        15

    accuracy                           0.90        60
   macro avg       0.91      0.90      0.90        60
weighted avg       0.91      0.90      0.90        60



/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
